# LSTM + LinearX, LinearY

Cite from https://github.com/jjbecomespheh/Trajectory_Prediction_Using_nuScenes_Dataset 

- This model uses basic LSTM, which use incremental prediction, in the iteration, it predicts the next point (x, y) of the trajectory in turn, and then use this new (x, y) as the training trajectory point to add to the sequence.

- We not cite their ADE and FDE result directly, because 
 - They used their own processed datasets, which is not fit our requirements. Such as they only use (x, y) rather than (x, y, v, a, r) (v: speed, a: acceleration, r: heading rate).
 - Their prediction output not met the requirements of nuScenes prediction task. The offcial made 6 second predictions at 2 Hz, n_timesteps is 12. So we need to predict 12 points of single trajectory. However, their output is 6 points.

- For the above reasons, so we reconstruct their part of the code to meet our requirements.

# Import 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# nuscenes-devkit tools 
!pip install nuscenes-devkit

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

import matplotlib.pyplot as plt
import tqdm

from nuscenes import NuScenes
from nuscenes.eval.prediction import metrics
from nuscenes.eval.prediction.data_classes import Prediction
from nuscenes.prediction import PredictHelper
import math

# Import data 

You can import the file "np_trajectory_data_22_with_VAR.npy" from [Link](https://drive.google.com/drive/folders/118Z18sWEg4CqHAhFcYmDqXvDj2qk4YtI?usp=sharing), it's the same with "trajectory" data in "DL-Project.ipynb" at code 3.3. Just for testing convenience 

`trajectory = get_trajectories(trajectory_data, uni_instance_ids)`


`np.save('np_trajectory_data_22_with_VAR.npy', trajectory)
`

In [ ]:
trajectory = np.load("/content/drive/MyDrive/DL_nuScenes/np_trajectory_data_22_with_VAR.npy", allow_pickle=True)

In [ ]:
trajectory.shape

(17503,)

In [ ]:
train_data = trajectory[0:int(len(trajectory)*0.6)]
val_data = trajectory[int(len(trajectory)*0.6): int(len(trajectory)*0.8)]
test_data = trajectory[int(len(trajectory)*0.8):]

# Using GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Utils

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    predicted_ls = []
    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        predicted = data[i+seq_length]
        x.append(_x)
        predicted_ls.append(predicted)
    return np.array(x), np.array(predicted_ls)

In [ ]:
# This method is to convert X and Y values into tensors and reshape them
def conv_and_reshape(x, y):

    X = torch.tensor(x, dtype=torch.float32)
    Y = torch.tensor(y, dtype=torch.float32)

    if len(X.shape) == 1:
        X = torch.reshape(X, (X.shape[0],1))
        Y = torch.reshape(Y, (Y.shape[0],1))
    elif len(X.shape) == 2:
        X = torch.reshape(X, (X.shape[0],X.shape[1],1))
        Y = torch.reshape(Y, (Y.shape[0],Y.shape[1],1))
    return X, Y

In [ ]:
def get_data(trajectory, seq_length):
    tra = []
    for t in trajectory:
        tra.append(t[2:4])
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    xy = scaler.fit_transform(tra)
    
    
    x = xy[:, 0]
    y = xy[:, 1]
    
    x, exp_x = sliding_windows(x, seq_length)
    y, exp_y = sliding_windows(y, seq_length)
    
    dataX, dataY = conv_and_reshape(x,y)
    exp_x, exp_y = conv_and_reshape(exp_x, exp_y)
    
    return dataX, dataY, exp_x, exp_y

# Model

In [ ]:
class LSTM_0_1(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_0_1, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Tensors in (batch, seq, feature)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_size, 1)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, data):
        h_0 = Variable(torch.zeros(
            self.num_layers, data.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, data.size(0), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        out, (_, _) = self.lstm(data, (h_0, c_0))

        out = out[:,-1,:]
        
        out_x = self.fc1(out)
        out_y = self.fc2(out)

        return out_x, out_y

###### Model instance

In [ ]:
lstm_0_1 = LSTM_0_1(2, 2, 1)
lstm_0_1 = lstm_0_1.to(device)

In [ ]:
learning_rate_0_1 = 0.001
loss_func_0_1 = torch.nn.MSELoss()
optimizer_0_1 = torch.optim.Adam(lstm_0_1.parameters(), lr=learning_rate_0_1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_0_1, step_size=3, gamma=0.1)

###### train function

In [ ]:
def clean_nan(tra):
    tra_new = []
    for t in tra:
        if math.isnan(t[4]) or math.isnan(t[5]) or math.isnan(t[6]):
            continue
        else:
            tra_new.append(t)
    return tra_new

In [ ]:
def train_0(trajectory, lstm, seq_length, epoch, optimizer, loss_func):
    loss_plot = []
    lstm.train()

    log_interval = 1000
    train_loss = 0
    rep_loss = 0 

    for i, tra in enumerate(trajectory):
        tra = clean_nan(tra)
        if len(tra) < 20 or tra[0][-1] == 0:  # 
            continue
        optimizer.zero_grad()
        dx, dy, ex, ey = get_data(tra, seq_length)
        cat_data = torch.cat([dx, dy], dim = 2)

        len_dx = len(dx)

        #GPU
        cat_data = cat_data.to(device)
        label = torch.tensor([float(tra[0][-1])]*len_dx).to(device)

        predicted_output_x, predicted_output_y = lstm(cat_data)
        ex = ex.to(device)
        ey = ey.to(device)
        loss_x = loss_func(predicted_output_x, ex)
        loss_y = loss_func(predicted_output_y, ey)

        loss = loss_x + loss_y
        train_loss += loss
        rep_loss += loss

        loss.backward()
        optimizer.step()

        if i % log_interval == 0 and i > 0:
            print(f"| Epoch {epoch:3d} | {i:5d}/{len(trajectory):5d} batches | loss: {rep_loss/log_interval:8.3f}")
            rep_loss = 0
    return train_loss/len(trajectory)
        

###### train

In [ ]:
t_0 = []
seq_length = 8
for epoch in range(6):
    train_loss_0 = train_0(train_data, lstm_0_1, seq_length, epoch, optimizer_0_1, loss_func_0_1)
    t_0.append(train_loss_0.item())

| Epoch   0 |  1000/10501 batches | loss:    0.417
| Epoch   0 |  2000/10501 batches | loss:    0.237
| Epoch   0 |  4000/10501 batches | loss:    0.205
| Epoch   0 |  7000/10501 batches | loss:    0.149
| Epoch   0 |  8000/10501 batches | loss:    0.037
| Epoch   0 |  9000/10501 batches | loss:    0.033
| Epoch   1 |  1000/10501 batches | loss:    0.024
| Epoch   1 |  2000/10501 batches | loss:    0.022
| Epoch   1 |  4000/10501 batches | loss:    0.043
| Epoch   1 |  7000/10501 batches | loss:    0.062
| Epoch   1 |  8000/10501 batches | loss:    0.023
| Epoch   1 |  9000/10501 batches | loss:    0.022
| Epoch   2 |  1000/10501 batches | loss:    0.019
| Epoch   2 |  2000/10501 batches | loss:    0.017
| Epoch   2 |  4000/10501 batches | loss:    0.037
| Epoch   2 |  7000/10501 batches | loss:    0.055
| Epoch   2 |  8000/10501 batches | loss:    0.022
| Epoch   2 |  9000/10501 batches | loss:    0.021


### Prediction

###### look_ahead_test

In [ ]:
def look_ahead_test(lstm, trajectory, seq_length, frames_to_predict, start):
    traj_xy = []  
    for t in trajectory:
        traj_xy.append(t[2:4])
    scaler = MinMaxScaler(feature_range=(-1, 1))
    xy = scaler.fit_transform(traj_xy)
    xs = xy[:,0]
    ys = xy[:,1]
    x, expected_x = sliding_windows(xs, seq_length)
    y, expected_y = sliding_windows(ys, seq_length)
    x, expected_x = [x[start]], expected_x[start:]
    y, expected_y = [y[start]], expected_y[start:]

    original_x, original_y = xs[:start+seq_length], ys[:start+seq_length]

    dataX, dataY = conv_and_reshape(x,y)

    original_x, original_y = conv_and_reshape(original_x, original_y)
    expected_x, expected_y = conv_and_reshape(expected_x, expected_y)


    lstm.eval()

    with torch.no_grad():
        predicted_data = []
        cat_data = torch.cat([dataX, dataY], dim=2)
        cat_data = cat_data.to(device)
        for i in range(frames_to_predict):

            predicted_output_x, predicted_output_y = lstm(cat_data)
            combined_predicted_output = torch.cat([predicted_output_x, predicted_output_y], dim=1)

            new_cat_data = cat_data[0][1:]

            new_data = torch.cat([new_cat_data, combined_predicted_output])
            cat_data = torch.reshape(new_data, (1,new_data.shape[0], new_data.shape[1]))
            
            combined_predicted_output = combined_predicted_output.cpu().data
            data_predict = scaler.inverse_transform(combined_predicted_output)
            predicted_data.append(data_predict[0])

        combined_original_traj = torch.cat([original_x, original_y], dim=1)
        combined_expected_traj = torch.cat([expected_x, expected_y], dim=1)

        data_original = scaler.inverse_transform(combined_original_traj)
        data_expected = scaler.inverse_transform(combined_expected_traj)

        pred_data = np.reshape(predicted_data, (len(predicted_data),len(predicted_data[0])))
    return pred_data, data_expected, data_original



In [ ]:
frames_to_predict = 12
p_one_mode = np.array([[1.]])


def traj2modes(trajs):
    return np.array([trajs])


def My_metrics_with_nunsceces(lstm, test_data, seq_length, frames_to_predict, start):
    results_look_ahead = {}
    avg_ade = 0
    avg_fde = 0
    avg_missRate = 0
    length = len(test_data)
    
    for tra in tqdm.tqdm(test_data):
        tra = clean_nan(tra)
        if len(tra) < 20: 
            length -= 1
            continue
        if tra[0][-1] == 0:
            continue
        
        predicted_data, data_expected, data_original = look_ahead_test(lstm, tra, seq_length, frames_to_predict, start)
        
        modes = data_expected.shape[0]
        if modes > 12:
            data_expected = data_expected[0:12]
        elif modes < 12:
            predicted_data = predicted_data[0:modes]
        x_one_mode, y_one_mode = traj2modes(predicted_data), traj2modes(data_expected)

        ade = metrics.min_ade_k(x_one_mode, y_one_mode, p_one_mode)
        fde = metrics.min_fde_k(x_one_mode, y_one_mode, p_one_mode)
        missRate = metrics.miss_rate_top_k(x_one_mode, y_one_mode, p_one_mode, 2)


        avg_ade += ade
        avg_fde += fde
        avg_missRate += missRate
    avg_ade = avg_ade/length
    avg_fde = avg_fde/length
    avg_missRate = 1 - avg_missRate/length
    print()
    print("avg_ade:",avg_ade[0][0])
    print("avg_fde:",avg_fde[0][0])
    print("avg_missRate:",avg_missRate[0][0])

In [ ]:
My_metrics_with_nunsceces(lstm_0_1, test_data, seq_length, frames_to_predict, 0)

  0%|          | 0/3501 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  after removing the cwd from sys.path.
100%|██████████| 3501/3501 [00:13<00:00, 261.25it/s]


avg_ade: 4.026117876916457
avg_fde: 7.337618028983116
avg_missRate: 0.6660949113779302
